In [ ]:
from defense.generate_dataset import generate_defense_set, generate_testset
import tensorflow as tf
import pandas as pd
import numpy as np
from art.estimators.classification import TensorFlowV2Classifier
from evaluation_metric import evaluate_metric

# TRAIN DETECTOR

In [10]:
# from defense.generate_dataset import generate_dataset

samples = pd.read_csv("NSL-KDD/samples/samples.csv").drop(columns=["Unnamed: 0"])
y_samples = pd.read_csv("NSL-KDD/samples/y_samples.csv").drop(columns=["Unnamed: 0"])

fsgm_samples = pd.read_csv("NSL-KDD/samples/attack/FSGM_sample.csv").drop(columns=["Unnamed: 0"])
bim_samples = pd.read_csv("NSL-KDD/samples/attack/BIM_sample.csv").drop(columns=["Unnamed: 0"])
cwl2_samples = pd.read_csv("NSL-KDD/samples/attack/CW-L2_sample.csv").drop(columns=["Unnamed: 0"])
jsma_samples = pd.read_csv("NSL-KDD/samples/attack/JSMA_sample.csv").drop(columns=["Unnamed: 0"])
deepfool_samples = pd.read_csv("NSL-KDD/samples/attack/DeepFool_sample.csv").drop(columns=["Unnamed: 0"])

full_features = samples.columns
shap_importance_df = pd.read_csv("NSL-KDD/samples/shap_importance.csv").drop(columns=["Unnamed: 0"])
selected_features = shap_importance_df.loc[shap_importance_df['shap_importance'] > 0]['column_name'].values
shap_importance_feature_values = shap_importance_df.loc[shap_importance_df['shap_importance'] > 0]['shap_importance'].values

In [11]:
from imblearn.over_sampling import SMOTE

defense_data = generate_defense_set(
    samples,
    fsgm_samples,
    bim_samples,
    cwl2_samples,
    jsma_samples,
    deepfool_samples,
    selected_features,
    shap_importance_feature_values,
    40
)
from sklearn.model_selection import train_test_split
x_defense_train, x_defense_test, y_defense_train, y_defense_test = train_test_split(
    defense_data.drop(columns=['label']),
    defense_data['label'],
    test_size=0.3,
    stratify=defense_data['label']
)

balance_defense = SMOTE(sampling_strategy={0:1750})

x_defense_train, y_defense_train = balance_defense.fit_resample(x_defense_train, y_defense_train)

x_defense_train = np.reshape(x_defense_train, (x_defense_train.shape[0], x_defense_train.shape[1], 1))
x_defense_test = np.reshape(x_defense_test, (x_defense_test.shape[0], x_defense_test.shape[1], 1))
y_defense_train = pd.get_dummies(y_defense_train)*1
y_defense_test = pd.get_dummies(y_defense_test)*1

In [12]:
def create_cnn(n_features):
    ae_detector_dnn = tf.keras.Sequential(
        layers=[
            tf.keras.layers.Convolution1D(2048, 3, padding='same', activation='relu', input_shape=(n_features,1)),
            tf.keras.layers.MaxPooling1D(pool_size=4),
            tf.keras.layers.Convolution1D(512, 3, padding='same', activation='relu'),
            tf.keras.layers.MaxPooling1D(pool_size=4),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dense(2, activation='softmax'),
        ]
    )
    
    ae_detector_dnn.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['accuracy']
    )
    
    return ae_detector_dnn

ae_detector_cnn = create_cnn(x_defense_train.shape[1])
ae_detector_cnn.fit(x_defense_train, y_defense_train, epochs=100, batch_size=16)

Epoch 1/100
219/219 [==============================] - 17s 71ms/step - loss: 0.4775 - accuracy: 0.7731
Epoch 2/100
219/219 [==============================] - 16s 71ms/step - loss: 0.3847 - accuracy: 0.8434
Epoch 3/100
219/219 [==============================] - 15s 69ms/step - loss: 0.3598 - accuracy: 0.8520
Epoch 4/100
219/219 [==============================] - 15s 67ms/step - loss: 0.3457 - accuracy: 0.8571
Epoch 5/100
219/219 [==============================] - 15s 68ms/step - loss: 0.3334 - accuracy: 0.8643
Epoch 6/100
219/219 [==============================] - 15s 69ms/step - loss: 0.3206 - accuracy: 0.8689
Epoch 7/100
219/219 [==============================] - 14s 62ms/step - loss: 0.3270 - accuracy: 0.8691
Epoch 8/100
219/219 [==============================] - 13s 57ms/step - loss: 0.3096 - accuracy: 0.8751
Epoch 9/100
219/219 [==============================] - 13s 59ms/step - loss: 0.3070 - accuracy: 0.8746
Epoch 10/100
219/219 [==============================] - 13s 58ms/step - l

# EVALUATE

RQ1: on test-set from defense data

In [13]:
evaluate_metric(y_pred=ae_detector_cnn.predict(x_defense_train), y_true=y_defense_train)

110/110 [==============================] - 1s 12ms/step


{'confusion_matrix': array([[1724,   26],
        [ 270, 1480]]),
 'accuracy_score': 0.9154285714285715,
 'precision_score': 0.9154285714285715,
 'recall_score': 0.9154285714285715,
 'f1_score': 0.9154285714285715}

In [14]:
evaluate_metric(y_pred=ae_detector_cnn.predict(x_defense_test), y_true=y_defense_test)

29/29 [==============================] - 0s 13ms/step


{'confusion_matrix': array([[118,  32],
        [151, 599]]),
 'accuracy_score': 0.7966666666666666,
 'precision_score': 0.7966666666666666,
 'recall_score': 0.7966666666666666,
 'f1_score': 0.7966666666666665}

RQ2: on random test set

In [ ]:
dnn = tf.keras.models.load_model("NSL-KDD/dnn.h5")

X_train = pd.read_csv("NSL-KDD/X_train.csv").drop(columns=['Unnamed: 0'])
X_test = pd.read_csv("NSL-KDD/X_test.csv").drop(columns=['Unnamed: 0'])
y_test = pd.read_csv("NSL-KDD/y_test.csv").drop(columns=['Unnamed: 0'])


loss_object = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
dnn_model = TensorFlowV2Classifier(
    model=dnn,
    loss_object=loss_object,
    optimizer=optimizer,
    nb_classes=5,
    input_shape=X_train.shape
)

test_samples = X_test.sample(n=500)
y_test_samples = y_test.iloc[test_samples.index]
ae_data, label_ae_data = generate_testset(
    dnn_model,
    test_samples,
    y_test_samples,
    selected_features,
    40
)
ae_data.to_csv("NSL-KDD/defense/test/data.csv")
pd.DataFrame(label_ae_data, columns=['label']).to_csv("NSL-KDD/defense/test/y_data.csv")

In [16]:
evaluate_metric(y_pred=ae_detector_cnn.predict(ae_data[selected_features[:40]]), y_true=pd.get_dummies(label_ae_data)*1)

16/16 [==============================] - 0s 15ms/step


{'confusion_matrix': array([[ 78,   6],
        [365,  51]]),
 'accuracy_score': 0.258,
 'precision_score': 0.258,
 'recall_score': 0.258,
 'f1_score': 0.258}